In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

In [3]:
vectorstore=FAISS.from_texts(
  texts=["小明在华为工作","熊喜欢吃蜂蜜"],
  embedding=embeddings
)
vectorstore

In [4]:
retriever=vectorstore.as_retriever()

In [5]:
retriever.invoke("熊喜欢吃什么")

[Document(metadata={}, page_content='熊喜欢吃蜂蜜'),
 Document(metadata={}, page_content='小明在华为工作')]

In [6]:
from langchain_openai import ChatOpenAI,OpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt=ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.runnables import RunnablePassthrough,RunnableParallel
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval=RunnableParallel(
    {
      "context": retriever,
      "question": RunnablePassthrough()
    }
)

chain=setup_and_retrieval | prompt | model | StrOutputParser()

In [9]:
chain.invoke("小明在哪里工作?")

'华为'

In [10]:
#完整链式
chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
chain.invoke("小明在哪里工作?")

'小明在华为工作。'

In [17]:
from operator import itemgetter

template="""
只根据以下文档回答问题：
{context}

问题：{question}
回答问题请加上称呼"{name}",并且用逗号隔开。
"""

prompt=ChatPromptTemplate.from_template(template)

chain=(
    {"context":itemgetter("question") | retriever,
     "question":itemgetter("question"),
     "name":itemgetter("name")
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question":"小明在哪里工作?","name":"主人"})


'主人在华为工作。'